In [ ]:
%cd drive/MyDrive/Poems
%ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install hazm

In [ ]:
import glob
import torch
import pickle
import numpy as np
import pandas as pd
from hazm import word_tokenize
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, CrossEncoder, util

# def get_word_vector(sent, tokenizer, model, layers):
#     """Get a word vector by first tokenizing the input sentence, getting all token idxs
#     that make up the word of interest, and then `get_hidden_states`."""
#     encoded = tokenizer.encode_plus(sent, return_tensors="pt")
#     with torch.no_grad():
#         output = model(**encoded)
#      # Get all hidden states
#     states = output.hidden_states
#      # Stack and sum all requested layers
#     output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
#     # get all token idxs that belong to the word of interest
#     word_nums = len(sent.split(" "))
#     embeddings = []
#     for idx in range(word_nums):
#         token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
#         word_tokens_output = output[token_ids_word].mean(dim=0)
#         embeddings.append(word_tokens_output)
#     return embeddings

# def get_query_similarity(query, tfidf, tfidf_array, wv_tfidf):
#     query_vector = tfidf.transform([get_cleaned_text(query)]).toarray() @ wv_tfidf
#     dot_product = tfidf_array * query_vector
#     similarity = np.sum(dot_product, axis = 1)
#     return similarity

#########################################################################################################################

def bert(query, num_results):
    data = pd.read_csv('./bert_data/corpus.csv')
    data = data.beyt.tolist()
    
    # encoder = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
    # encoder.save_pretrained('./bert_models/SimpleBert')
    encoder = SentenceTransformer('./bert_models/SimpleBert')

    # corpus_embeddings = encoder.encode(data, convert_to_tensor=True, show_progress_bar=True, batch_size=256)
    # corpus_embeddings = corpus_embeddings.cpu().detach().numpy()
    # np.save('./bert_data/simple_embeddings.npy', corpus_embeddings)
    corpus_embeddings = torch.from_numpy(np.load('./bert_data/simple_embeddings.npy', allow_pickle=True))
    
    query_embedding = encoder.encode(query, convert_to_tensor=True).to('cpu')
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0, :]
    cos_scores = cos_scores.cpu().detach().numpy()
    most_relevant = np.argsort(cos_scores)[::-1]
    results = []
    for i in range(num_results):
        results.append((most_relevant[i], data[most_relevant[i]]))
    return results

def bert_cross(query, num_results):
    data = pd.read_csv('./bert_data/corpus.csv')
    data = data.beyt.tolist()
    encoder = SentenceTransformer('./bert_models/SimpleBert')
    corpus_embeddings = torch.from_numpy(np.load('./bert_data/simple_embeddings.npy'))
    query_embedding = encoder.encode(query, convert_to_tensor=True).to('cpu')
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=10)
    hits = hits[0]
    cross_encoder = CrossEncoder('./bert_models/CrossEncoder')
    cross_inp = [[query, data[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
    re_ranked = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    results = []
    for i in range(num_results):
        results.append((re_ranked[i]['corpus_id'], data[re_ranked[i]['corpus_id']]))
    return results

def bert_finetuned(query, num_results):
    data = pd.read_csv('./bert_data/corpus.csv')
    data = data.beyt.tolist()
    encoder = SentenceTransformer('./bert_models/MyBert')
    
    # corpus_embeddings = encoder.encode(data, convert_to_tensor=True, show_progress_bar=True, batch_size=256)
    # corpus_embeddings = corpus_embeddings.cpu().detach().numpy()
    # np.save('./bert_data/mybert_embeddings.npy', corpus_embeddings)
    corpus_embeddings = torch.from_numpy(np.load('./bert_data/mybert_embeddings.npy'))
    
    query_embedding = encoder.encode(query, convert_to_tensor=True).to('cpu')
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0, :]
    cos_scores = cos_scores.cpu().detach().numpy()
    most_relevant = np.argsort(cos_scores)[::-1]
    results = []
    for i in range(num_results):
        results.append((most_relevant[i], data[most_relevant[i]]))
    return results

# def bert_weighted_sum(query, num_results):
#     data = pd.read_csv('./bert_data/corpus.csv')
#     data = data.beyt.tolist()
#     tokenizer = AutoTokenizer.from_pretrained('./bert_models/SimpleBert', local_files_only=True)
#     # tokenizer = word_tokenize
#     model = AutoModel.from_pretrained('./bert_models/SimpleBert', output_hidden_states=True, local_files_only=True)
#     vectorizer_tfidf = TfidfVectorizer(norm='l2', tokenizer=word_tokenize, max_features=30000)

#     layers = [-1]
#     sent = "ایشان هر روز ماهی میل می کردند ."
#     word_embedding = get_word_vector(sent, tokenizer, model, layers)
#     print(len(word_embedding))
#     print(word_embedding[0].shape)
#     return
    
#     X = vectorizer_tfidf.fit_transform(data)
#     with open('./bert_data/X', 'wb') as f:
#         pickle.dump(X, f)
#     # with open('./bert_data/X', 'rb') as f:
#     #     X = pickle.load(f)
#     print(type(X))
#     print(X.shape)

#     print(vectorizer_tfidf.get_feature_names())
#     word_embedding_tfidf = get_word_vector(vectorizer_tfidf.get_feature_names(), tokenizer, model, layers)
#     X_ft = X.toarray() @ word_embedding_tfidf
#     doc_similarity = get_query_similarity(query, vectorizer_tfidf, X_ft, word_embedding_tfidf)
#     relevance_order = np.argsort(doc_similarity)[::-1]
#     results = []
#     for i in range(num_results):
#         results.append(data[relevance_order[i]])
#     return results

In [ ]:
query = 'خورشید و یار'
k = 10

# results = bert(query, k)
# results = bert_cross(query, k)
results = bert_finetuned(query, k)
# results = bert_weighted_sum(query, k)

results

# **Evaluation**

In [ ]:
# rel_ids = {i+1:[] for i in range(100)}
# data_pd = pd.read_csv('./bert_data/corpus.csv')
# for qid in range(1, 100):
#     print(qid, end=' ')
#     with open('./evaluation_data/relevant/{}.txt'.format(qid), 'r') as f:
#         lines = f.readlines()
#         lines = [line[:-1] for line in lines]
#     for line in lines:
#         x = data_pd[data_pd['beyt'] == line].index.to_numpy()
#         try:
#             rel_ids[qid].append(x[0])
#         except:
#             pass
#     # assert len(rel_ids[qid]) == len(lines)
# rel_ids


In [27]:
# with open('rel_ids.pickle', 'wb') as handle:
#     pickle.dump(rel_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

rel_ids = None
with open('rel_ids.pickle', 'rb') as handle:
    rel_ids = pickle.load(handle)

In [ ]:
# queries_paths = './evaluation_data/eval_queries.txt'
mutated_paths = './evaluation_data/eval_s.csv'
ks = [1, 21, 41, 61, 81, 101]

In [ ]:
def percision_at_k(rel, ret):
    rel_set = set(rel)
    ret_set = set(ret)
    intersection = rel_set.intersection(ret_set)
    return len(intersection) / len(ret)

In [ ]:
# First and Third model

encoder = SentenceTransformer('./bert_models/MyBert')
corpus_embeddings = torch.from_numpy(np.load('./bert_data/mybert_embeddings.npy', allow_pickle=True)).to('cuda:0')

mutated_queries = pd.read_csv(mutated_paths)
data_pd = pd.read_csv('./bert_data/corpus.csv')
data = data_pd.beyt.tolist()

for k in ks:
    pers = []
    for i, row in mutated_queries.iterrows():
        q = row['text']
        qid = int(row['original_query_id'])

        query_embedding = encoder.encode(q, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0, :]
        cos_scores = cos_scores.cpu().detach().numpy()
        most_relevant = np.argsort(cos_scores)[::-1]
        results = []
        for j in range(k):
            results.append((most_relevant[j], data[most_relevant[j]]))

        returned_ids = [j for j, _ in results]
        
        rids = rel_ids[qid]
        # with open('./evaluation_data/relevant/{}.txt'.format(qid), 'r') as f:
        #     lines = f.readlines()
        #     lines = [line[:-1] for line in lines]
        # for line in lines:
        #     x = data_pd[data_pd['beyt'] == line].index.to_numpy()
        #     rel_ids.append(x[0])
        # assert len(rel_ids) == len(lines)

        p_at_k = percision_at_k(rids, returned_ids)
        pers.append(p_at_k)

    print('Precision at {}:'.format(k), sum(pers) / len(pers))

In [ ]:
# Second model

encoder = SentenceTransformer('./bert_models/SimpleBert')
cross_encoder = CrossEncoder('./bert_models/CrossEncoder')
corpus_embeddings = torch.from_numpy(np.load('./bert_data/simple_embeddings.npy', allow_pickle=True)).to('cuda:0')

mutated_queries = pd.read_csv(mutated_paths)
data_pd = pd.read_csv('./bert_data/corpus.csv')
data = data_pd.beyt.tolist()

for k in ks:
    pers = []
    for i, row in mutated_queries.iterrows():
        # print(i, end=' ')
        q = row['text']
        qid = int(row['original_query_id'])

        query_embedding = encoder.encode(q, convert_to_tensor=True)
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=100)
        hits = hits[0]
        cross_inp = [[q, data[hit['corpus_id']]] for hit in hits]
        cross_scores = cross_encoder.predict(cross_inp)
        for idx in range(len(cross_scores)):
            hits[idx]['cross-score'] = cross_scores[idx]
        re_ranked = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
        most_relevant = np.argsort(cos_scores)[::-1]
        results = []
        for j in range(k):
            results.append((most_relevant[j], data[most_relevant[j]]))

        returned_ids = [j for j, _ in results]
        
        rids = rel_ids[qid]
        # with open('./evaluation_data/relevant/{}.txt'.format(qid), 'r') as f:
        #     lines = f.readlines()
        #     lines = [line[:-1] for line in lines]
        # for line in lines:
        #     x = data_pd[data_pd['beyt'] == line].index.to_numpy()
        #     rel_ids.append(x[0])
        # assert len(rel_ids) == len(lines)

        p_at_k = percision_at_k(rids, returned_ids)
        pers.append(p_at_k)

    print('Precision at {}:'.format(k), sum(pers) / len(pers))

# **Temp**

In [ ]:
encoder = AutoModel.from_pretrained('HooshvareLab/bert-fa-zwnj-base')
tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-fa-zwnj-base')
encoder.save_pretrained('./bert_models/SimpleBert')
tokenizer.save_pretrained('./bert_models/SimpleBert')
tokenizer.save_pretrained('./bert_models/MyBert')

In [ ]:
encoder = AutoModel.from_pretrained("m3hrdadfi/bert-fa-base-uncased-wikinli")
tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/bert-fa-base-uncased-wikinli")
encoder.save_pretrained('./bert_models/CrossEncoder')
tokenizer.save_pretrained('./bert_models/CrossEncoder')

In [ ]:
l = []
with open('./evaluation_data/eval_queries.txt', 'r') as f:
    lines = f.readlines()
    lines = [s[:-1] for s in lines]
    for i, line in enumerate(lines):
        l.append({'original_query_id':i+1, 'text': line})
df = pd.DataFrame(l)
df.to_csv('./evaluation_data/eval_s.csv')